# Plant Recognition - Inference with Quantized TFLite Model
This notebook demonstrates how to run inference on new images 
using the quantized EfficientNetB0 TFLite model.

In [1]:
# Core libraries
import os
import cv2
import numpy as np
import tensorflow as tf
import json
import matplotlib.pyplot as plt
from tensorflow.keras.applications.efficientnet import preprocess_input


## Configuration


In [2]:
MODEL_PATH = 'models/model_quant.tflite'
IMG_SIZE = (224, 224)
TEST_DIR = 'data/new_test_images'

# ## Load class names
with open('data/processed/class_indices.json', 'r') as f:
    class_indices = json.load(f)
    CLASS_NAMES = list(class_indices.keys())

print(f"Loaded {len(CLASS_NAMES)} classes: {CLASS_NAMES}")

FileNotFoundError: [Errno 2] No such file or directory: 'data/processed/class_indices.json'

## Load TFLite model


In [ ]:
interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("\nModel input details:", input_details[0])
print("Model output details:", output_details[0])

## Prediction function

In [ ]:
def predict_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return "Error: Could not read image", 0.0, None
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, IMG_SIZE)
    input_data = preprocess_input(np.expand_dims(img_resized, axis=0).astype(np.float32)).astype(np.uint8)
    
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    # Dequantize output
    scale, zero_point = output_details[0]['quantization']
    output_data = (output_data.astype(np.float32) - zero_point) * scale
    
    predicted_idx = np.argmax(output_data)
    confidence = output_data[0][predicted_idx]
    
    class_name = CLASS_NAMES[predicted_idx] if predicted_idx < len(CLASS_NAMES) else f"Unknown class {predicted_idx}"
    return class_name, confidence, img

Found 3658 images belonging to 9 classes.
Found 785 images belonging to 9 classes.
Found 785 images belonging to 9 classes.


## Run inference on test images

## Model Definition


In [ ]:
if not os.path.exists(TEST_DIR):
    os.makedirs(TEST_DIR)
    print(f"Created test directory: {TEST_DIR}")
    print("Please add some test images (JPG/PNG) and rerun.")
else:
    valid_files = [f for f in os.listdir(TEST_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    if not valid_files:
        print("No test images found. Add JPG/PNG files to the test directory.")
    else:
        plt.figure(figsize=(15, 10))
        
        for i, filename in enumerate(valid_files):
            img_path = os.path.join(TEST_DIR, filename)
            class_name, confidence, img = predict_image(img_path)
            
            plt.subplot(2, 3, i+1)
            plt.imshow(img)
            plt.title(f"{filename}\nPred: {class_name}\nConf: {confidence*100:.1f}%", fontsize=10)
            plt.axis('off')
            
            print(f"{filename} -> {class_name} ({confidence:.4f})")
        
        plt.tight_layout()
        plt.savefig('models/new_images_test.png')
        plt.show()